In [0]:
BASE_PATH = "~/drive/colaboratory/Hacktuit/"

In [0]:
!wget -nc http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip glove.840B.300d.zip

--2018-07-22 17:18:33--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2018-07-22 17:18:34--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip]
Saving to: ‘glove.840B.300d.zip’

glove.840B.300d.zip  18%[==>                 ] 387.57M  9.33MB/s    eta 2m 39s 

glove.840B.300d.zip 100%[===================>]   2.03G  17.8MB/s    in 3m 21s  

2018-07-22 17:21:56 (10.3 MB/s) - ‘glove.840B.300d.zip’ saved [2176768927/2176768927]

Archive:  glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


In [0]:
!pip install nltk
!pip install tqdm
!pip install keras_tqdm
!pip install ipywidgets

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 51kB 4.9MB/s 
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 112kB 6.6MB/s 
    100% |████████████████████████████████| 2.2MB 12.5MB/s 
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [0]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from nltk.tokenize import TweetTokenizer
import keras
from keras import layers
from keras import backend as K
from keras_tqdm import TQDMNotebookCallback
from IPython import display

Using TensorFlow backend.


In [0]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    try:
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
    except:
      print ("Ignoring word: ", word)
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [03:19, 10985.12it/s]

Found 2196016 word vectors.


In [0]:
train_df = pd.read_csv(BASE_PATH + 'inshorts_train.csv')
train_df['Category'] = train_df['Category'].map({
    "business" : 0,
    "entertainment" : 1,
    "national" : 2,
    "politics" : 3,
    "science" : 4,
    "sports" : 5,
    "technology": 6,
    "world" : 7
})
test_df = pd.read_csv(BASE_PATH + 'inshorts_test.csv')
concat_df = test_df.append(train_df)
print (train_df.shape)
print (test_df.shape)
print (concat_df.shape)

train_df.head()

(64253, 5)
(16064, 4)
(80317, 5)


,ID,Date,Title,Article,Category
0,528,"05 Apr 2018,Thursday",4 bank employees booked in alleged scam of â‚¹...,The Rajasthan police has registered a case aga...,0
1,51112,"08 Dec 2017,Friday","Gayle hits 45-ball ton, becomes first to reach...",Windies' Chris Gayle smashed a 45-ball hundred...,5
2,73666,"28 Oct 2017,Saturday",Taiwan holds Asia's largest gay pride parade,Taiwan on Saturday held Asia's largest gay pri...,7
3,13719,"07 Dec 2017,Thursday",Arjun slams reports of man assaulting him on f...,Arjun Kapoor has slammed reports of a man assa...,1
4,17356,"30 Jun 2017,Friday","Maybe, I'm the most spoilt superstar but Iove ...","Speaking on his stardom, actor Shah Rukh Khan ...",1


In [0]:
from keras.preprocessing.sequence import pad_sequences 

MAX_SEQUENCE_LENGTH = 200

vocabulary = dict()
inverse_vocabulary = ['PADDING']

def generate_sequence(text_list):
    tknzr = TweetTokenizer(reduce_len=True)
  
    sequences = []
    
    for text in text_list:
        #text = text.split()
        text = tknzr.tokenize(text.lower())
        text_sequence = []
        for word in text:
            if word not in vocabulary:
                vocabulary[word] = len(inverse_vocabulary)
                text_sequence.append(len(inverse_vocabulary))
                inverse_vocabulary.append(word)
            else:
                text_sequence.append(vocabulary[word])
        sequences.append(text_sequence)
    print("%d unique tokens in the vocabulary" %len(vocabulary))

    return sequences

concat_seq = generate_sequence(concat_df.Article.values.tolist())

97364 unique tokens in the vocabulary


In [0]:
from sklearn.model_selection import train_test_split

train_seq = generate_sequence(train_df.Article.values.tolist())
bodies_seq = pad_sequences(train_seq, maxlen=MAX_SEQUENCE_LENGTH)
labels = train_df.Category.values.tolist()

data = {
    'train': {},
    'val': {},
    'test': {},
}

(data['train']['text'],
 data['val']['text'],
 data['train']['y'],
 data['val']['y']
) = train_test_split(bodies_seq, labels, test_size=0.3, random_state=42)

(data['val']['text'],
 data['test']['text'],
 data['val']['y'],
 data['test']['y']
) = train_test_split(data['val']['text'], data['val']['y'], test_size=0.1, random_state=42)

print('Train: {} examples'.format(len(data['train']['y'])))
print('Val: {} examples'.format(len(data['val']['y'])))
print('Test: {} examples'.format(len(data['test']['y'])))

97364 unique tokens in the vocabulary
Train: 44977 examples
Val: 17348 examples
Test: 1928 examples


In [0]:
EMBEDDING_DIM = 300

embedding_matrix = np.zeros((len(vocabulary) + 1, EMBEDDING_DIM)) 
for word, i in vocabulary.items():
    if word in embeddings_index.keys():
        embedding_matrix[i] = embeddings_index[word]

print (embedding_matrix.shape)

embedding_layer = layers.Embedding(
    len(vocabulary) + 1,
    EMBEDDING_DIM,
    weights=[embedding_matrix],
    input_length=MAX_SEQUENCE_LENGTH,
    name='Embedding',
    trainable=False
)

(97365, 300)


In [0]:
from keras.layers import concatenate,Input,Dense, GlobalAveragePooling1D, Embedding, Dropout

class CnnModel(keras.models.Model):
    
    def __init__(self, filter_sizes, weights_matrix=embedding_matrix):
        replicate_count = 10
        projection_dropout = 0.5

        input_layer = layers.Input(
            shape=(MAX_SEQUENCE_LENGTH,),
            name='Input'
        )
        
        eb_layer = embedding_layer(input_layer)
                
        conv_layers = []
        for filter_size in filter_sizes:
            for _r in range(replicate_count):
                str_id = '_' + str(filter_size) + '_' + str(_r)
                conv = layers.Conv1D(
                    EMBEDDING_DIM, 
                    filter_size, 
                    activation='relu',
                    name='Conv1D-' + str_id
                )(eb_layer)
                max_pool = layers.GlobalMaxPooling1D(
                    name='MaxPool' + str_id
                )(conv)
                dropout  = layers.Dropout(
                    rate=projection_dropout
                )(max_pool)
                conv_layers.append(dropout)

        concatenated_layer = concatenate(
            conv_layers,
            name='ConcatenatedRegions'
        )
        
        fc_layer = layers.Dense(
            units=10,
            name='FullyConnected'
        )(concatenated_layer)
        
        dropout_layer = layers.Dropout(
            rate=projection_dropout
        )(fc_layer)
             
        predictions = layers.Dense(
            8,
            activation='softmax',
            name='Output'
        )(dropout_layer)
        
        super().__init__(inputs=[input_layer], outputs=predictions)
          
    def compile(self):
        return super().compile(
            optimizer=keras.optimizers.Adam(lr=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

In [0]:
class LstmModel(keras.models.Model):
    
    def __init__(self, weights_matrix=embedding_matrix):
        projection_dropout = 0.5

        input_layer = layers.Input(
            shape=(MAX_SEQUENCE_LENGTH,),
            name='Input'
        )
        
        eb_layer = embedding_layer(input_layer)

        # spatial_dropout_layer = layers.SpatialDropout1D(0.4)(eb_layer)
        
        bilstm_layer = layers.Bidirectional(
            layers.LSTM(
                units=200,
                dropout=0.5, 
                recurrent_dropout=0.3,
            )
        )(eb_layer)
        
        fc_layer = layers.Dense(
            units=10,
            activation='relu',
            name='FullyConnected'
        )(bilstm_layer)
        
        dropout_layer = layers.Dropout(
            rate=projection_dropout
        )(fc_layer)
        
        predictions = layers.Dense(
            8,
            activation='softmax',
            name='Output'
        )(dropout_layer)
        
        super().__init__(inputs=[input_layer], outputs=predictions)
        
    def compile(self):
        return super().compile(
            optimizer=keras.optimizers.Adam(lr=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

In [0]:
class GruModel(keras.models.Model):
    
    def __init__(self, weights_matrix=embedding_matrix):
        projection_dropout = 0.5

        input_layer = layers.Input(
            shape=(MAX_SEQUENCE_LENGTH,),
            name='Input'
        )
        
        eb_layer = embedding_layer(input_layer)

        # spatial_dropout_layer = layers.SpatialDropout1D(0.4)(eb_layer)
        
        bilstm_layer = layers.GRU(
            units=200,
            dropout=0.5, 
            recurrent_dropout=0.3
        )(eb_layer)
        
        fc_layer = layers.Dense(
            units=50,
            activation='relu',
            name='FullyConnected'
        )(bilstm_layer)
        
        dropout_layer = layers.Dropout(
            rate=projection_dropout
        )(fc_layer)
        
        predictions = layers.Dense(
            8,
            activation='softmax',
            name='Output'
        )(dropout_layer)
        
        super().__init__(inputs=[input_layer], outputs=predictions)
        
    def compile(self):
        return super().compile(
            optimizer=keras.optimizers.Adam(lr=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

In [0]:
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import LabelBinarizer

!mkdir models-10



def fit_model(model, epochs, batch_size, prefix=""):
    filepath = BASE_PATH[2:] + "models-10/" + prefix + "-{epoch:02d}.hdf5"
    encoder = LabelBinarizer()

    history = model.fit(
        [data['train']['text']],
        keras.utils.to_categorical(data['train']['y'], 8),
        epochs=epochs,
        verbose=1,
        callbacks=[
            ModelCheckpoint(
                filepath, 
                verbose=1,
                save_best_only=True
            )
        ],
        validation_data=(
            [data['val']['text']],
            keras.utils.to_categorical(data['val']['y'], 8),
        ),
        batch_size=batch_size,
    )
    return history

mkdir: cannot create directory ‘models-10’: File exists


In [0]:
def generate_models():
    model = CnnModel([1, 2, 3])
    model.compile()
    history = fit_model(model, 8, 512, 'CNN-123')

    model = CnnModel([3, 4, 5])
    model.compile()
    history = fit_model(model, 30, 512, 'CNN-345')

    model = LstmModel()
    model.compile()
    history = fit_model(model, 30, 512, 'BLSTM')
    
    model = GruModel()
    model.compile()
    history = fit_model(model, 50, 1024, 'GRU')
generate_models()

Train on 44977 samples, validate on 17348 samples
Epoch 1/8
44977/44977 [==============================] - 268s 6ms/step - loss: 2.0610 - acc: 0.5144 - val_loss: 0.6212 - val_acc: 0.8168

Epoch 00001: val_loss improved from inf to 0.62118, saving model to drive/colaboratory/Hacktuit/models-10/CNN-123-01.hdf5
Epoch 2/8
44977/44977 [==============================] - 256s 6ms/step - loss: 1.1142 - acc: 0.6479 - val_loss: 0.5733 - val_acc: 0.8209

Epoch 00002: val_loss improved from 0.62118 to 0.57332, saving model to drive/colaboratory/Hacktuit/models-10/CNN-123-02.hdf5
Epoch 3/8
37888/44977 [========================>.....] - ETA: 34s - loss: 0.9727 - acc: 0.6735

44977/44977 [==============================] - 257s 6ms/step - loss: 0.9619 - acc: 0.6768 - val_loss: 0.5523 - val_acc: 0.8303

Epoch 00003: val_loss improved from 0.57332 to 0.55226, saving model to drive/colaboratory/Hacktuit/models-10/CNN-123-03.hdf5
Epoch 4/8
44977/44977 [==============================] - 257s 6ms/step - loss: 0.8485 - acc: 0.7106 - val_loss: 0.5864 - val_acc: 0.8157

Epoch 00004: val_loss did not improve from 0.55226
Epoch 5/8
44977/44977 [==============================] - 256s 6ms/step - loss: 0.7863 - acc: 0.7264 - val_loss: 0.5420 - val_acc: 0.8110

Epoch 00005: val_loss improved from 0.55226 to 0.54198, saving model to drive/colaboratory/Hacktuit/models-10/CNN-123-05.hdf5
Epoch 6/8
11264/44977 [======>.......................] - ETA: 2:46 - loss: 0.7256 - acc: 0.7433

44977/44977 [==============================] - 257s 6ms/step - loss: 0.7025 - acc: 0.7503 - val_loss: 0.5201 - val_acc: 0.8203

Epoch 00006: val_loss improved from 0.54198 to 0.52010, saving model to drive/colaboratory/Hacktuit/models-10/CNN-123-06.hdf5
Epoch 7/8
44977/44977 [==============================] - 257s 6ms/step - loss: 0.6563 - acc: 0.7641 - val_loss: 0.5367 - val_acc: 0.8210

Epoch 00007: val_loss did not improve from 0.52010
Epoch 8/8
44977/44977 [==============================] - 257s 6ms/step - loss: 0.6134 - acc: 0.7806 - val_loss: 0.5394 - val_acc: 0.8219

Epoch 00008: val_loss did not improve from 0.52010


In [0]:
from google.colab import files

from os import listdir
import random, re
import numpy as np
from keras.models import load_model
from sklearn.ensemble import VotingClassifier

def shouldProcess(s):
    if 'CNN-' not in s and 'LSTM-' not in s and 'GRU' not in s: return False

    epoch = int(re.findall('-([0-9]*).hdf5', s)[0])
    if 'CNN-345' in s and epoch > 3: return True
    if 'LSTM-' in s and epoch > 14: return True
    if 'BLSTM-' in s and epoch > 14: return True
    if 'GRU-' in s and epoch > 15: return True
    return False

model_files = [ str(BASE_PATH[2:] + 'models-10/' + x) for x in listdir(BASE_PATH[2:] + 'models-10/') if shouldProcess(x)]

print (model_files)

random.seed(42)
random.shuffle(model_files)

rand_models_files = model_files[:30]

models = []
for file in rand_models_files:
    print (file)
    m = None
    if 'CNN-123' in file:
        m = CnnModel([1, 2, 3])
    elif 'CNN-345' in file:
        m = CnnModel([3, 4, 5])
    elif 'LSTM-' in file:
        m = LstmModel()
    elif 'GRU-' in file:
        m = GruModel()
    m.load_weights(file)
    models.append((file, m))

print (models)

['drive/colaboratory/Hacktuit/models-10/GRU-27.hdf5', 'drive/colaboratory/Hacktuit/models-10/BLSTM-26.hdf5', 'drive/colaboratory/Hacktuit/models-10/CNN-123-06.hdf5']
drive/colaboratory/Hacktuit/models-10/BLSTM-26.hdf5
drive/colaboratory/Hacktuit/models-10/GRU-27.hdf5
drive/colaboratory/Hacktuit/models-10/CNN-123-06.hdf5
[('drive/colaboratory/Hacktuit/models-10/BLSTM-26.hdf5', <__main__.LstmModel object at 0x7f513dba6e48>), ('drive/colaboratory/Hacktuit/models-10/GRU-27.hdf5', <__main__.GruModel object at 0x7f51313d1860>), ('drive/colaboratory/Hacktuit/models-10/CNN-123-06.hdf5', <__main__.CnnModel object at 0x7f512a090550>)]


In [0]:
from keras.models import Model

def ensembleModels(models, model_input):
    # collect outputs of models in a list
    yModels=[model(model_input) for model in models] 
    # averaging outputs
    yAvg=layers.average(yModels) 
    # build model from same input and avg output
    modelEns = Model(inputs=model_input, outputs=yAvg,    name='ensemble')  
   
    return modelEns

model_input = Input(shape=(MAX_SEQUENCE_LENGTH,)) # c*h*w
modelEns = ensembleModels([m[1] for m in models], model_input)
modelEns.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
lstmmodel_20 (LstmModel)        (None, 8)            30015198    input_3[0][0]                    
__________________________________________________________________________________________________
grumodel_14 (GruModel)          (None, 8)            29520558    input_3[0][0]                    
__________________________________________________________________________________________________
cnnmodel_33 (CnnModel)          (None, 8)            34708598    input_3[0][0]                    
__________________________________________________________________________________________________
average_3 

In [0]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

# h = np.sum([m[1].predict([data['test']['text']]) for m in models], 0)

h = modelEns.predict([data['val']['text']])

print (h.shape)

predict = np.asarray([np.argmax(i) for i in h])
gold = np.asarray(data['val']['y'])
resultf1 = accuracy_score(gold, predict)
print('Accuracy score on {} test samples: {:.6}'.format(
    len(predict), resultf1
))

print (metrics.confusion_matrix(gold, predict))
print (metrics.classification_report(gold, predict))

(17348, 8)
Accuracy score on 17348 test samples: 0.843728
[[1912   19  108   28   10   11  179   95]
 [  13 2132   20   53    1   57   28   33]
 [ 170   87 1266  542   13   25   26  109]
 [  15   31   94 2129    0    6    2    7]
 [   5    5    9    1 1101    2   36   32]
 [   2   38    4    3    5 2234    9   16]
 [ 185   21   22    0  109   14 1870   82]
 [  70   32   68    7   47   24   81 1993]]
             precision    recall  f1-score   support

          0       0.81      0.81      0.81      2362
          1       0.90      0.91      0.91      2337
          2       0.80      0.57      0.66      2238
          3       0.77      0.93      0.84      2284
          4       0.86      0.92      0.89      1191
          5       0.94      0.97      0.95      2311
          6       0.84      0.81      0.82      2303
          7       0.84      0.86      0.85      2322

avg / total       0.84      0.84      0.84     17348



In [0]:
# Generate labels for challenge set
test_seq = generate_sequence(test_df.Article.values.tolist())
test_seq = pad_sequences(test_seq, maxlen=MAX_SEQUENCE_LENGTH)

print (len(test_seq))

# test_h = np.sum([m[1].predict([test_seq]) for m in models], 0)
test_h = modelEns.predict([test_seq])

97364 unique tokens in the vocabulary
16064


In [0]:
outfile = open('test_out.csv', 'w')
outfile.write("ID,business,entertainment,national,politics,science,sports,technology,world\n")
for id, col in zip(test_df.ID.values.tolist(), test_h):
  #  print (str(id) + ',' + ','.join([format(e, 'f') for e in col]))
    outfile.write(str(id) + ',' + ','.join([format(e, 'f') for e in col]) + '\n')
outfile.close()

In [0]:
from google.colab import files
files.download('test_out.csv')